In [27]:
from openai import OpenAI
from PyPDF2 import PdfReader
import json
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def generate_response(prompt):
    completion = client.chat.completions.create(
        model="NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q4_0.gguf",
        messages=[
            {"role": "system", "content": "For each question,option pair extract the question content, options content and output them in a JSON format,JSON must not contain answers,only question content and respective option content. if a question has question then option statements you msut include them in the JSON as well."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
    )
    return completion.choices[0].message.content

def generate_answer_response(prompt):
    completion = client.chat.completions.create(
        model="NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q4_0.gguf",
        messages=[
            {"role": "system", "content": "extract the question number, correct option and some important context of the answer in form of JSON, where keys are question,option,context ."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
    )
    return completion.choices[0].message.content

def parse_pdf(questions_pdf, answers_pdf):
    # Read the questions PDF content
    with open(questions_pdf, 'rb') as file:
        questions_reader = PdfReader(file)
        questions_content = ''
        for page in questions_reader.pages:
            questions_content += page.extract_text()

    # Read the answers PDF content
    with open(answers_pdf, 'rb') as file:
        answers_reader = PdfReader(file)
        answers_content = ''
        for page in answers_reader.pages:
            answers_content += page.extract_text()

    # Remove the instructions from the questions content
    instructions_end = questions_content.find("DO NOT OPEN THIS BOOKLET UNTIL YOU ARE ASKED TO DO SO")
    questions_content = questions_content[instructions_end + len("DO NOT OPEN THIS BOOKLET UNTIL YOU ARE ASKED TO DO SO"):].strip()

    # Remove the "www.visionias.in ©Vision IAS" text from the questions content
    questions_content = questions_content.replace("www.visionias.in ©Vision IAS", "")

    return questions_content, answers_content

def process_questions(questions_content):
    chunk_size = 2048
    output_json = ""

    for i in range(0, len(questions_content), chunk_size):
        chunk = questions_content[i:i+chunk_size]
        response = generate_response(chunk)
        output_json += response

    return output_json

def process_answers(answers_content):
    chunk_size = 2048
    output_json = ""

    for i in range(0, len(answers_content), chunk_size):
        chunk = answers_content[i:i+chunk_size]
        response = generate_answer_response(chunk)
        output_json += response
    print('running')
    return output_json


def save_json_to_file(json_data, file_path):
    try:
        # Attempt to parse the JSON data
        parsed_data = json.loads(json_data)
        
        # If parsing is successful, write the parsed data to the file
        with open(file_path, 'w') as file:
            json.dump(parsed_data, file, indent=2)
        
        print(f"JSON data saved successfully to {file_path}")
    
    except json.JSONDecodeError as e:
        # Handle the JSONDecodeError exception
        print(f"Error: Invalid JSON data - {str(e)}")
        print("Please check the JSON data and try again.")


q, a = parse_pdf('q.pdf', 'a.pdf')

output_json = process_questions(q)
output_answers_json = process_answers(a)
# Save the JSON output to a file
save_json_to_file(json.loads(output_json), 'output.json')
save_json_to_file(json.loads(output_answers_json), 'answers_output.json')


KeyboardInterrupt: 